# Data cleaning and feature engeneering

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import sys
sys.path.insert(0,'..')
from utils.utils import create_date, drop_smart_contract, clean_up_row

import networkx as nx
import operator

## Import July 2017 data

In [5]:
!nvidia-smi

Sun Jul 23 17:56:25 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:3B:00.0 Off |                  Off |
|  0%   36C    P8               34W / 260W|     10MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
# Test data Sept'2017

eth_06092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_06092017.csv")
eth_07092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_07092017.csv")
eth_08092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_08092017.csv")
eth_09092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_09092017.csv")
eth_10092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_10092017.csv")
eth_11092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_11092017.csv")
eth_12092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_12092017.csv")
eth_13092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_13092017.csv")
eth_14092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_14092017.csv")
eth_15092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_15092017.csv")

/tmp/ipykernel_1775327/1757000105.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_09092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_09092017.csv")
/tmp/ipykernel_1775327/1757000105.py:8: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_11092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_11092017.csv")
/tmp/ipykernel_1775327/1757000105.py:9: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_12092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_12092017.csv")
/tmp/ipykernel_1775327/1757000105.py:10: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  eth_13092017 = pd.read_csv("https://s3.eu-central-2.wasabisys.com/ethblockchain/eth_13092017.csv")
/tmp/ipykernel_1775327/17570001

In [7]:
!nvidia-smi

Sun Jul 23 17:57:37 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Quadro RTX 8000                 Off| 00000000:3B:00.0 Off |                  Off |
|  0%   36C    P8               35W / 260W|     10MiB / 49152MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
data = pd.concat([eth_06092017,eth_07092017, eth_08092017,
                  eth_09092017,eth_10092017,eth_11092017,
                  eth_12092017,eth_13092017,eth_14092017,
                  eth_15092017])

# Missing values check

In [9]:
data.isna().sum()

hash                                 0
nonce                                0
transaction_index                    0
from_address                         0
to_address                       17881
value                                0
gas                                  0
gas_price                            0
input                                0
receipt_cumulative_gas_used          0
receipt_gas_used                     0
receipt_contract_address       3802215
receipt_root                         0
receipt_status                 3820096
block_timestamp                      0
block_number                         0
block_hash                           0
max_fee_per_gas                3820096
max_priority_fee_per_gas       3820096
transaction_type               3820096
receipt_effective_gas_price          0
dtype: int64

# Data cleaning

In order to clean the data the following steps need to be performed:
- remove invalid transactions using 'clean_up_row' using the hash which should always be 0x hash
- drop smart contracts: smart contracts which are not proper transactions (receipt_contract_address not null correspond to smart contracts)
- drop when from and to address is missing & 'block_timestamp' is missing
- drop columns (as are missing attributes) : receipt_status, max_fee_per_gas, max_priority_fee_per_gas, transaction_type     
- drop duplicated transactions
- create dates (day, month, year, date)
- transform data of 'object' type into numeric ('float')
                 


In [10]:
# Remove invalid transaction
data = clean_up_row(data)

# Remove smart contracts
data_no_smart = drop_smart_contract(data)

# Drop when from and to address is missing & 'block_timestamp' is missing
data_no_miss = drop_missing_data(data_no_smart)

# Remove duplicated rows
data_no_smart_nodup_nomiss = data_no_miss.drop_duplicates()

# Create dates 
data = create_date(data_no_smart_nodup_nomiss)

# transform attributes form 'object' dtype into numeric
data['gas'] = data['gas'].astype('float')
data['value'] = data['value'].astype('float')


### check if there are missing values

In [11]:
data_no_miss.isna().sum()

hash                           0
nonce                          0
transaction_index              0
from_address                   0
to_address                     0
value                          0
gas                            0
gas_price                      0
input                          0
receipt_cumulative_gas_used    0
receipt_gas_used               0
receipt_root                   0
block_timestamp                0
block_number                   0
block_hash                     0
receipt_effective_gas_price    0
dtype: int64

## Feature engineering and Feature transformation

### Additional/transformed features

1. Compute the gas fees per unit: 'gas_price_unit'
2. Compute the value of the transaction divided by the gas: 'value'
3. Number of transactions per address form/to
4. Number of transactions in the block
5. Degree centrality: degree centrality of the addresses form and addresses to, in degree centrality of addresses to and from and out degree centrality of addresses to and form


In [12]:
#1. Compute the gas fees per unit: 'gas_price_unit'
data['gas_price_unit'] = data['gas_price']/data['gas']

#2. Compute the value of the transaction divided by the gas: 'value'
data['value_div_gas'] = data['value']/data['gas']

#3. Number of transactions per address from and to

# count number of transaction for each 'address from' in July 2017
data['from_address_count'] = data.groupby('from_address')['from_address'].transform('count')
# count number of transaction for each 'address to' in July 2017
data['to_address_count'] = data.groupby('to_address')['to_address'].transform('count')

#4. Number of transactions per block
data['block_count'] = data.groupby('block_number')['block_number'].transform('count')

In [13]:
data.head(10)

,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,input,receipt_cumulative_gas_used,...,date,dates,year,month,day,gas_price_unit,value_div_gas,from_address_count,to_address_count,block_count
399999,0x116731b9bbd756a87bd14e0d717ee5bf3b011bcfe3c7...,0,38,0x4223db6fa17f18bd59ac2f6048d9f7433e3dee77,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,2.395800e+17,21000.0,20000000000,0x,2059821,...,2017-09-15,2017-09-15,2017,9,15,952380.952381,1.140857e+13,1,29901,112
133968,0x6cc3454d9fd46f3e9db18d9ed325673797b1b48a5447...,1602890,57,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0xd94e685ae0497a5c247ef2f7c3462c75bdcf7272,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,2382062,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,4,157
133946,0xbe5849cb35690538d2971b7367f14cc7161ebaa34d41...,1600807,56,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x28b1d974532976f43a904945d2bcdcaac4af00a9,0.000000e+00,211965.0,51000000000,0x6ea056a9000000000000000000000000b64ef51c8889...,2510448,...,2017-09-15,2017-09-15,2017,9,15,240605.760385,0.000000e+00,191290,76,132
133947,0x001c9fd121699611e3bf8c880f35391a25d849c5f3ce...,1591653,175,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0xf265bcd226b3f67d92cce6a353ad188b06e6670f,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,6418587,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,12,172
133948,0xd4680b73a45af8899edd37baec6fb7f9ca732c8e348d...,1591652,129,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x9c0bf39a11991fe7d3098191cd8fba70c847644e,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,2997622,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,8,172
133949,0x1f2ea9925081e62128ccf3899431bea9194619f4dd2b...,1591660,35,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x7514ab845745f2503e7635be3bb96409d53a7b5a,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,4642945,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,21,59
133950,0x6c0c847d2aae3815aff8c16f07eb42989d744c37ee48...,1591659,26,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x795c7e7cbbeed34bec82df971fb99848271c781a,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,4103934,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,2,59
133951,0x59e974a522de945cced3ccac09d2f3092a71c6ad808f...,1611116,28,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x575d2e446c5ecff8d4c6b8fa25c632dc904f68eb,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,952841,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,6,151
133952,0x29d377556098aafea0f4bf695736f13dedb9dcc7142a...,1611115,27,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x74dc84ffc1acc66eece75f00b01b40c26aef4b2f,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,902848,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,4,151
133953,0xcb3abcf4f74b35dfa54b265de9b1fab20b439534fb2e...,1604816,27,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0xca4b32c83a0f194dc7cbcd18ceafcfc15f6c3be9,0.000000e+00,150270.0,51000000000,0x6ea056a9000000000000000000000000000000000000...,1243472,...,2017-09-15,2017-09-15,2017,9,15,339389.099621,0.000000e+00,191290,6,138


### 5. Degree centrality of the addresses to and from

In [14]:
# Create the graph using network x
df_networkx = data[['from_address','to_address']]
print('df length', len(df_networkx))

df_networkx_nodup = df_networkx.drop_duplicates()
print('df address only length',len(df_networkx_nodup))

transaction_address = list(zip(df_networkx.from_address,df_networkx.to_address))
print(len(transaction_address))

df length 3802215


df address only length 1908340
3802215


Compute adjacency matrix and degree centrality of a multidirected graph

In [15]:
# DIRECTED GRAPH / MULTI-DIRECTED GRAPH

#G = nx.DiGraph()
G = nx.MultiDiGraph()
G.add_edges_from(transaction_address)
print(G)


MultiDiGraph with 1053653 nodes and 3802215 edges


In [16]:
degree_centrality=nx.degree_centrality(G)

In [17]:
# sort the degree centrality
degree_centrality_sort = dict(sorted(degree_centrality.items(), key=operator.itemgetter(1), reverse=True))
items = degree_centrality_sort.items()
len(degree_centrality)

1053653

In [18]:
degree_centrality_from = pd.DataFrame({'from_address': [i[0] for i in items], 'degree_centrality_from': [i[1] for i in items]})
degree_centrality_to = pd.DataFrame({'to_address': [i[0] for i in items], 'degree_centrality_to': [i[1] for i in items]})

in_degree_to = pd.DataFrame(G.in_degree, columns=['to_address','in_degree_adr_to'])
out_degree_to = pd.DataFrame(G.out_degree,columns=['to_address','out_degree_adr_to'])

in_degree_from = pd.DataFrame(G.in_degree, columns=['from_address','in_degree_adr_from'])
out_degree_from = pd.DataFrame(G.out_degree,columns=['from_address','out_degree_adr_from'])

In [19]:
degree_centrality_to.head()

,to_address,degree_centrality_to
0,0x8d12a197cb00d4747a1fe03395095ce2a5cc6819,0.232230
1,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0.181554
2,0x829bd824b016326a401d083b33d092293333a830,0.128800
3,0xea674fdde714fd979de3edf0f56aa9716b898ec8,0.107216
4,0x69ea6b31ef305d6b99bb2d4c9d99456fa108b02a,0.102207


In [20]:
# Merge the degree centrality to the transaction data

df = data.merge(degree_centrality_from, how='left', on='from_address')
df = df.merge(degree_centrality_to, how='left', on='to_address')

df = df.merge(in_degree_to, how='left', on='to_address')
df = df.merge(out_degree_to, how='left', on='to_address')

df = df.merge(in_degree_from, how='left', on='from_address')
df = df.merge(out_degree_from, how='left', on='from_address')

## Add labels : malicious addresses

This are the malicious addresses detected, github 

url = "https://raw.githubusercontent.com/MyEtherWallet/ethereum-lists/master/src/addresses/addresses-darklist.json"

Description:
Added new address that holds phishing/stolen funds
Contains the following fields:

- address: can be address to or from 
- comment: description of the malicous transaction/attack
- date: transaction date

### Malicious address cleaning:

this list has been clean as dates were intially in a different fromat

In [21]:
darklist = pd.read_csv('darklist.csv',index_col=False, usecols=["address", "comment", "date;"])
darklist.head()

,address,comment,date;
0,0xDdd6854A002A6fbcDF695385cD5ed630c9E27C3e,Scam address contacted Trust Wallet users in T...,2018-10-25;
1,0x21918461C6aecA5EAEc825B4746D64a0D4028dF6,Scam address of phishing email sent to leaked ...,2017-09-05;
2,0xd821dfb705333d2c16e0c0c0be75ce360cca566a,BitJob token sale scam site.,2017-09-12;
3,0x66817272D39Da7fd4C552F430fC0B694e357c157,Comments on Etherscan are reporting funds bein...,2017-07-19;
4,0x36db23fd8e0fabb300ba95bfdf4a0f20b05eebfb,FAKE_Tezos,;


In [22]:
darklist.isna().sum()

address     16
comment    246
date;       16
dtype: int64

### add malicous addresses

In [23]:
# Merge malicoous addresses on the 'from_address'
df = df.merge(darklist, how='left', left_on='from_address', right_on='address')
df.loc[df['address'].notna(), 'mal_trans_from'] = 1
df.drop(['address',],axis=1,inplace=True)
df.rename(columns = {'comment':'comment_from_address_darklst'}, inplace = True)

In [24]:
# Merge malicoous addresses on the 'to_address'
df = df.merge(darklist, how='left', left_on='to_address', right_on='address')
df.loc[df['address'].notna(), 'mal_trans_to'] = 1
df.drop(['address',],axis=1,inplace=True)
df.rename(columns = {'comment':'comment_to_address_darklst'}, inplace = True)

In [25]:
df.isna().sum()

hash                                  0
nonce                                 0
transaction_index                     0
from_address                          0
to_address                            0
value                                 0
gas                                   0
gas_price                             0
input                                 0
receipt_cumulative_gas_used           0
receipt_gas_used                      0
receipt_root                          0
block_timestamp                       0
block_number                          0
block_hash                            0
receipt_effective_gas_price           0
block_timestamp_str                   0
date                                  0
dates                                 0
year                                  0
month                                 0
day                                   0
gas_price_unit                        0
value_div_gas                         0
from_address_count                    0


In [26]:
df.rename(columns = {'date;_x':'date_mal_trans_from','date;_y':'date_mal_trans_to'}, inplace = True)

In [27]:
# Check how many times there is an overlap
df['transaction_flag'] = 0
df.loc[df['mal_trans_from'].notna(),'transaction_flag']=1
df.loc[df['mal_trans_to'].notna(),'transaction_flag']=1
df['transaction_flag'].value_counts()


transaction_flag
0    3801992
1        249
Name: count, dtype: int64

In [28]:
anomalous_trans = df['transaction_flag'].value_counts()[1]
print(f'Total number of transaction : {len(df)}, total number of anomalous transactions: {anomalous_trans} ({anomalous_trans/len(df)*100})')

Total number of transaction : 3802241, total number of anomalous transactions: 249 (0.006548769528286082)


In [29]:
df['transaction_flag1'] = 0
df['transaction_flag2'] = 0

df.loc[df['mal_trans_from'].notna(),'transaction_flag1']=1
df.loc[df['mal_trans_to'].notna(),'transaction_flag2']=1
df['transaction_flag1'].value_counts()

transaction_flag1
0    3802182
1         59
Name: count, dtype: int64

In [30]:
df['transaction_flag2'].value_counts()

transaction_flag2
0    3802051
1        190
Name: count, dtype: int64

In [31]:
df['double_trans_mal'] = df['transaction_flag1'] + df['transaction_flag2']
df['double_trans_mal'].value_counts()

double_trans_mal
0    3801992
1        249
Name: count, dtype: int64

For these 35 transaction below both the address to and the address form are flagged as malicious, however the comment is not always the same. the assumption is that is more likely to be received by the malicious address rather than the send by it, so in these corner cases the comment related to the 'to_address' would be considered.
Some attack dates and commetns are missing, the merge happened based on the address from and to

In [32]:
df['attack_descr'] = df['comment_to_address_darklst'].where(pd.notnull, df['comment_from_address_darklst'])

In [33]:
df['attack_date'] = df['date_mal_trans_to'].where(pd.notnull, df['date_mal_trans_from'])

In [34]:
df.columns

Index(['hash', 'nonce', 'transaction_index', 'from_address', 'to_address',
       'value', 'gas', 'gas_price', 'input', 'receipt_cumulative_gas_used',
       'receipt_gas_used', 'receipt_root', 'block_timestamp', 'block_number',
       'block_hash', 'receipt_effective_gas_price', 'block_timestamp_str',
       'date', 'dates', 'year', 'month', 'day', 'gas_price_unit',
       'value_div_gas', 'from_address_count', 'to_address_count',
       'block_count', 'degree_centrality_from', 'degree_centrality_to',
       'in_degree_adr_to', 'out_degree_adr_to', 'in_degree_adr_from',
       'out_degree_adr_from', 'comment_from_address_darklst',
       'date_mal_trans_from', 'mal_trans_from', 'comment_to_address_darklst',
       'date_mal_trans_to', 'mal_trans_to', 'transaction_flag',
       'transaction_flag1', 'transaction_flag2', 'double_trans_mal',
       'attack_descr', 'attack_date'],
      dtype='object')

In [35]:
X = df